In [ ]:
!pip install -q datasets pandas numpy

In [ ]:
import pandas
from datasets import load_dataset, load_metric
import torch
from transformers import BertForSequenceClassification, AutoTokenizer

data_files = {
    "train": "Aniemore/Text/main/train.csv",
    "validation": "Aniemore/Text/main/test.csv",
}

dataset = load_dataset("csv", data_files=data_files, delimiter="\t", )
train_dataset = dataset["train"]
eval_dataset = dataset["validation"]

train_dataframe = pandas.DataFrame(train_dataset)
test_dataframe = pandas.DataFrame(eval_dataset)

train_dataframe

Using custom data configuration default-8a1f5a0beb1300ca
Reusing dataset csv (C:\Users\lii29\.cache\huggingface\datasets\csv\default-8a1f5a0beb1300ca\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)
100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 401.08it/s]


,text,labels,source
0,Суровый гималайский медведь .,[],lj
1,"Так, обнаружено несколько проблем с дисплеем (...",[],lenta
2,У меня остается только один вопрос - является ...,[2],lj
3,Забавно как люди в возрасте удивляются входящи...,[0],twitter
4,"Издание The Register отмечает, что у владельце...",[],lenta
...,...,...,...
7523,Ухаха какая я злая!,[4],twitter
7524,Я очень рада вас видеть.,[0],twitter
7525,Жизнь боль.,[1],twitter
7526,- Не удивительно .,[],lj


In [ ]:
LABELS = ['neutral', 'happiness', 'sadness', 'enthusiasm', 'fear', 'anger', 'disgust']

In [ ]:
train_dataframe.labels.apply(str).value_counts()

[]        3043
[0]       1508
[1]       1365
[2]        568
[3]        553
[4]        384
[0, 2]      27
[5]         17
[1, 3]      12
[1, 0]       9
[0, 3]       6
[3, 0]       6
[0, 1]       5
[3, 1]       4
[3, 4]       4
[1, 2]       3
[3, 2]       3
[4, 0]       3
[2, 0]       2
[2, 1]       2
[0, 4]       2
[2, 3]       1
[2, 4]       1
Name: labels, dtype: int64

In [16]:
def binarize_labels(labels):
    return [int(len(labels)==0) ] + [int(i in labels) for i in range(6)]

print(binarize_labels([]))
print(binarize_labels([0]))
print(binarize_labels([1, 4]))

[1, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0]
[0, 0, 1, 0, 0, 1, 0]


In [17]:
rubert_model = 'cointegrated/rubert-tiny2'

tokenizer = AutoTokenizer.from_pretrained(rubert_model)
model = BertForSequenceClassification.from_pretrained(rubert_model, num_labels=7, problem_type='multi_label_classification')

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not 

In [18]:
!pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
model.cuda()

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, elemen

In [19]:
model.config.label2id = {label: _ for _, label in enumerate(LABELS)}
model.config.id2label = {_: label for _, label in enumerate(LABELS)}

In [20]:
cedr_mapped = dataset.map(lambda x: tokenizer(x['text'], truncation=True), batched=True).map(lambda x: {'label': [float(y) for y in binarize_labels(eval(x['labels']))]}, batched=False, remove_columns=['text', 'labels', 'source'])

Loading cached processed dataset at C:\Users\lii29\.cache\huggingface\datasets\csv\default-8a1f5a0beb1300ca\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-a18a88832912814d.arrow
Loading cached processed dataset at C:\Users\lii29\.cache\huggingface\datasets\csv\default-8a1f5a0beb1300ca\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-4eed0178a8fcd129.arrow
Loading cached processed dataset at C:\Users\lii29\.cache\huggingface\datasets\csv\default-8a1f5a0beb1300ca\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-34f093d372d16c67.arrow
Loading cached processed dataset at C:\Users\lii29\.cache\huggingface\datasets\csv\default-8a1f5a0beb1300ca\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-7f7545271fd32218.arrow


In [101]:
train_dev = cedr_mapped['train'].train_test_split(test_size=300, seed=42)
train_dev

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 7228
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 300
    })
})

In [102]:
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader
import gc

data_collator = DataCollatorWithPadding(tokenizer)

In [103]:
def cleanup():
    gc.collect()
    torch.cuda.empty_cache()

cleanup()

In [104]:
from tqdm.auto import tqdm, trange
from sklearn.metrics import roc_auc_score, precision_recall_fscore_support
from IPython.display import display
import numpy

def evaluate_model(model, dev_dataloader, verbose=False, labels=None):
    facts, preds = predict_with_model(model, dev_dataloader)
    aucs = get_classification_report(facts, preds, labels)
    if verbose:
        print('aucs:', aucs, numpy.mean(aucs))
    return numpy.mean(aucs)

def predict_with_model(model, dataloader):
    preds = []
    facts = []

    for batch in tqdm(dataloader):
        facts.append(batch.labels.cpu().numpy())
        batch = batch.to(model.device)
        with torch.no_grad():
            pr = model(input_ids=batch.input_ids, attention_mask=batch.attention_mask, token_type_ids=batch.token_type_ids)
        preds.append(torch.softmax(pr.logits, -1).cpu().numpy())
    facts = numpy.concatenate(facts)
    preds = numpy.concatenate(preds)
    return facts, preds

def get_classification_report(facts, preds, labels=None):
    print(facts.shape, preds.shape)
    # pfrs = pd.DataFrame(dict(zip(['p', 'r', 'f', 's'], precision_recall_fscore_support(facts, preds.argmax(1)))))
    # aucs = [roc_auc_score(facts[:, i], preds[:, i]) for i in range(7)]
    aucs = roc_auc_score(facts, preds)
    return aucs

In [105]:
batch_size = 64

train_dataloader = DataLoader(
    train_dev['train'],
    batch_size=batch_size, drop_last=False, shuffle=True, num_workers=0, collate_fn=data_collator
)
dev_dataloader = DataLoader(
    train_dev['test'],
    batch_size=batch_size, drop_last=False, shuffle=True, num_workers=0, collate_fn=data_collator
)

In [106]:
evaluate_model(model, dev_dataloader, verbose=True, labels=LABELS)

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 13.77it/s]

(300, 7) (300, 7)
aucs: 0.4924974862745959 0.4924974862745959


0.4924974862745959

In [107]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-5)

gradient_acc_step = 1
window = 500
clean_step = 100
report_step = 10000
ewm_loss = 0

In [108]:
model.train()
cleanup()

In [112]:
for epoch in trange(25):
    tq = tqdm(train_dataloader)

    for i, batch in enumerate(tq):
        try:
            batch = batch.to(model.device)
            output = model(**batch)
            loss = output.loss
            loss.backward()
        except RuntimeError as e:
            print('error on step', i, e)
            loss = None
            cleanup()
            continue

        if i and i % gradient_acc_step == 0:
            optimizer.step()
            optimizer.zero_grad()

        if i % clean_step == 0:
            cleanup()

        w = 1 / min(i+1, window)
        ewm_loss = ewm_loss * (1-w) + loss.item() * w
        tq.set_description(f'loss: {ewm_loss:4.4f}')

        if i % report_step == 0:
            model.eval()
            eval_loss = evaluate_model(model, dev_dataloader, verbose=True)
            model.train()
            print(f'epoch {epoch}, step {i}: train loss: {ewm_loss:4.4f}  val auc: {eval_loss}')

model.eval()
eval_loss = evaluate_model(model, dev_dataloader, verbose=True)
print(f'epoch {epoch + 1}, step {i}: train loss: {ewm_loss:4.4f}  val auc: {eval_loss}')

loss: 0.1772:   0%|                                                                            | 0/113 [00:00<?, ?it/s]

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 45.17it/s]

loss: 0.1867:   2%|█▏                                                                  | 2/113 [00:01<00:49,  2.24it/s]


(300, 7) (300, 7)
aucs: 0.8566920531437192 0.8566920531437192
epoch 0, step 0: train loss: 0.1772  val auc: 0.8566920531437192


loss: 0.1078:   0%|                                                                            | 0/113 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 65.58it/s]

loss: 0.1433:   4%|██▍                                                                 | 4/113 [00:00<00:11,  9.68it/s]


(300, 7) (300, 7)
aucs: 0.8600695882506547 0.8600695882506547
epoch 1, step 0: train loss: 0.1078  val auc: 0.8600695882506547


loss: 0.1516:   0%|                                                                            | 0/113 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 58.98it/s]

loss: 0.1308:   4%|██▍                                                                 | 4/113 [00:00<00:11,  9.52it/s]


...

(300, 7) (300, 7)
aucs: 0.86754359322118 0.86754359322118
epoch 23, step 0: train loss: 0.0290  val auc: 0.86754359322118


loss: 0.0367:   0%|                                                                            | 0/113 [00:00<?, ?it/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 62.67it/s]

loss: 0.0451:   3%|█▊                                                                  | 3/113 [00:00<00:14,  7.34it/s]

(300, 7) (300, 7)
aucs: 0.86508917648898 0.86508917648898
epoch 24, step 0: train loss: 0.0367  val auc: 0.86508917648898



100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 56.33it/s]

(300, 7) (300, 7)
aucs: 0.8720706332207219 0.8720706332207219
epoch 25, step 112: train loss: 0.0391  val auc: 0.8720706332207219


In [114]:
test_dataloader = DataLoader(
    cedr_mapped['validation'],
    batch_size=batch_size, drop_last=False, shuffle=True, num_workers=0, collate_fn=data_collator
)

In [115]:
model.eval()
eval_loss = evaluate_model(model, dev_dataloader, verbose=True)
test_loss = evaluate_model(model, test_dataloader, verbose=True)
model.train()
print(f'epoch {epoch}, step {i}: train loss: {ewm_loss:4.4f}  val auc: {eval_loss} test auc: {test_loss}')

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 12.08it/s]


(300, 7) (300, 7)
aucs: 0.8720706332207219 0.8720706332207219


100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 62.15it/s]

(1882, 7) (1882, 7)
aucs: 0.8560246372460613 0.8560246372460613
epoch 24, step 112: train loss: 0.0391  val auc: 0.8720706332207219 test auc: 0.8560246372460613


In [116]:
model.eval()
facts, preds = predict_with_model(model, test_dataloader)

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 39.09it/s]


In [117]:
model.save_pretrained('Text/Aniemore-fine-tuned-rubert-tiny2-cedr-emotion-detection')
tokenizer.save_pretrained('Text/Aniemore-fine-tuned-rubert-tiny2-cedr-emotion-detection')

('rubert-tiny2-cedr-emotion-detection\\tokenizer_config.json',
 'rubert-tiny2-cedr-emotion-detection\\special_tokens_map.json',
 'rubert-tiny2-cedr-emotion-detection\\vocab.txt',
 'rubert-tiny2-cedr-emotion-detection\\added_tokens.json',
 'rubert-tiny2-cedr-emotion-detection\\tokenizer.json')

In [127]:
@torch.no_grad()
def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return LABELS[predicted[0]]

In [147]:
model = model.to('cpu')
predict('Урааааа')

'happiness'